In [73]:
#https://python.langchain.com/v0.1/docs/use_cases/graph/constructing/

! pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j json-repair langsmith

In [74]:
import os
from langsmith import traceable

os.environ["LANGCHAIN_TRACING_V2"] = "true"
#langchain key is already set in .bashrc


In [75]:

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

#see additional env variables in the docker setup below from LC
#https://python.langchain.com/v0.1/docs/integrations/graphs/neo4j_cypher/

graph = Neo4jGraph()

In [76]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.chat_models.ollama import ChatOllama


#llm = ChatOllama(model="smangrul/llama-3-8b-instruct-function-calling")
llm = ChatOllama(model="phi3")

llm_transformer = LLMGraphTransformer(llm=llm)


In [ ]:
from langchain_core.documents import Document

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.  Joan Bonet is the first woman to win a gold medal in shotput.  She is best friends with Pierre Curie.
"""
#Joan Bonet is the first woman to win a gold medal in shotput.  She is best friends with Pierre Curie.
print("Loading document...")
documents = [Document(page_content=text)]
print("Document sucessfully loaded")

print("Converting document to neo4j graph...")
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print("Neo4j schema successfully created ")

print("Loading graph into neo4j")
graph.add_graph_documents(graph_documents)
print("Graph successfully loaded")
#print(f"Nodes:{graph_documents[0].nodes}")
#print(f"Relationships:{graph_documents[0].relationships}")




In [ ]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

In [ ]:

from langchain_community.document_loaders import TextLoader

loader = TextLoader("/home/jonot480/Documents/jon craig.txt")
doc_text = loader.load()
text=doc_text[0].page_content
documents = [Document(page_content=text)]#there may be an easier way to feed this, but i'm just using the langchain docs 
print(documents)



In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

graph_documents = llm_transformer.convert_to_graph_documents(documents)



In [ ]:
print(graph_documents)

In [ ]:
graph.add_graph_documents(graph_documents)

In [ ]:
llm_transformer_filtered = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Person", "Country", "Organization"],
    allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
)
graph_documents_filtered = llm_transformer_filtered.convert_to_graph_documents(
    documents
)
print(f"Nodes:{graph_documents_filtered[0].nodes}")
print(f"Relationships:{graph_documents_filtered[0].relationships}")

In [ ]:
#you need an LLM that can run function calling/tools
#open ai supports this, but you need something like nous hermes or llama3 
#phi3 doesn't seem to work 
#https://www.reddit.com/r/LocalLLaMA/comments/1cl6ocx/llm_with_function_calling_only_openai_models_have/

llm_transformer_props = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Person", "Country", "Organization"],
    allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
    node_properties=["born_year"],
)

graph_documents_props = llm_transformer_props.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents_props[0].nodes}")
print(f"Relationships:{graph_documents_props[0].relationships}")

graph.add_graph_documents(graph_documents_props)


